<a href="https://colab.research.google.com/github/PondKann/CXR-Project/blob/main/DenseNet201_Pretrained_BS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###### import

In [1]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
import pandas as pd
from torchsummary import summary
from skimage.io import imread, imsave
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from imutils import paths
import argparse

from sklearn.model_selection import KFold
from sklearn import metrics
from scipy.stats import zscore

In [3]:
import pickle
import joblib

######Load data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data_dir = '/content/drive/My Drive/Senior Project/CXR Image3class'

train 3 class

In [6]:
Normal151_dir = os.path.join(data_dir,'Normal151')
TB150_dir = os.path.join(data_dir,'TB150')
CA150_dir = os.path.join(data_dir,'CA150')

All_3class_dir = os.path.join(data_dir,'All_3class')

#### train 3 class  --- สร้างตารางเก็บ ชื่อภาพ และชื่อคลาส
ข้อมูลใน All_3class_dir รวมภาพทั้ง 3 คลาส รวม 451 ภาพ TB150, Normal151 และ CA150 (CA คือ lungcancer)

In [7]:
Normal151_dir = os.path.join(data_dir,'Normal151')
TB150_dir = os.path.join(data_dir,'TB150')
CA150_dir = os.path.join(data_dir,'CA150')

All_3class_dir = os.path.join(data_dir,'All_3class')

In [8]:
normal = []
for i in range(len(os.listdir(Normal151_dir))):
    img = os.listdir(Normal151_dir)[i]
    normal.append(img)
    
tnormal = pd.DataFrame({'ImageName':[normal][0],
                                       'NameType': "Normal" ,
                                       'NumberType': 0                      })

In [9]:
tuberculosis = []
for i in range(len(os.listdir(TB150_dir))):
    img = os.listdir(TB150_dir)[i]
    tuberculosis.append(img)

ttuberculosis = pd.DataFrame({'ImageName':[tuberculosis][0],
                                       'NameType': "Tuberculosis" ,
                                       'NumberType': 1                        })

In [10]:
lungcancer = []
for i in range(len(os.listdir(CA150_dir))):
    img = os.listdir(CA150_dir)[i]
    lungcancer.append(img)

tlungcancer = pd.DataFrame({'ImageName':[lungcancer][0],
                                       'NameType': "Lungcancer" ,
                                       'NumberType': 2                        })

In [11]:
data_train3 = pd.concat([tnormal, ttuberculosis, tlungcancer])
data_train3[-3:]

,ImageName,NameType,NumberType
147,ca26.jpg,Lungcancer,2
148,ca7.jpg,Lungcancer,2
149,ca8.jpg,Lungcancer,2


####  test 3 class จากเน็ต

In [ ]:
TB700_dir = os.path.join(data_dir,'test_model/Tuberculosis700')
Normal94_dir = os.path.join(data_dir,'test_model/NORMAL94')
CA150_dir = os.path.join(data_dir,'CA150')                  #อันเดิมที่ใช้เทรน

AllTest_2class = os.path.join(data_dir,'test_model/AllTest_2class')

In [ ]:
#สำหรับ test model
normal = []
for i in range(len(os.listdir(Normal94_dir))):
    img = os.listdir(Normal94_dir)[i]
    normal.append(img)
    
tnormal94 = pd.DataFrame({'ImageName':[normal][0],
                                       'NameType': "Normal" ,
                                       'NumberType': 0                      })

In [ ]:
#สำหรับ test model
tb700 = []
for i in range(len(os.listdir(TB700_dir))):
    img = os.listdir(TB700_dir)[i]
    tb700.append(img)

ttb700 = pd.DataFrame({'ImageName':[tb700][0],
                                       'NameType': "Tuberculosis" ,
                                       'NumberType': 1                        })

In [ ]:
#สำหรับ test model ข้อม฿ลเดิม
lungcancer = []
for i in range(len(os.listdir(CA150_dir))):
    img = os.listdir(CA150_dir)[i]
    lungcancer.append(img)

tlungcancer = pd.DataFrame({'ImageName':[lungcancer][0],
                                       'NameType': "Lungcancer" ,
                                       'NumberType': 2                        })

In [ ]:
data_test2 = pd.concat([tnormal94[:30], ttb700[:30], tlungcancer[:10]])

In [ ]:
test_NorTB = []
for ImageName in tqdm(data_test2['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test2
    image_path = AllTest_2class +'/' + ImageName                          # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (299, 299))
    test_NorTB.append(img)

100%|██████████| 70/70 [00:19<00:00,  3.60it/s]


In [ ]:
data_test = np.array(test_NorTB, dtype="float32") / 255.0

labels_test = np.array(data_test2[['NameType']] )        
labels_test.shape = (len(labels_test),) 

le3_Incep = LabelEncoder()                                                                     
labels_test = le3_Incep.fit_transform(labels_test)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_test = to_categorical(labels_test, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

## 1) K-fold  -- DenseNet201

### -->ACC = 0.70
### -->เวลา 3859.136 วินาที

*   BS = 8
*   activation="softmax"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))                                       #ขนาดของ input
    train_img_Dense.append(img)

data_Dense = np.array(train_img_Dense, dtype="float32") / 255.0

In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
INIT_LR = 1e-1
BS = 8
EPOCHS = 50

aug = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
base_model_Dense.trainable=False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Dense = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
kf_Dense = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Dense.split(data_Dense):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train_Dense = data_Dense[train]
    y_train_Dense = labels_Dense[train]
    x_test_Dense = data_Dense[test]
    y_test_Dense = labels_Dense[test]

    H_Dense = model_Dense.fit( x=aug.flow(x_train_Dense, y_train_Dense, batch_size=BS),
	                                            validation_data=(x_test_Dense, y_test_Dense),
	                                            steps_per_epoch=len(x_train_Dense) // BS,
	                                            epochs=30,  callbacks = [callback])
    
    pred_Dense = model_Dense.predict(x_test_Dense)
    
    oos_y.append(y_test_Dense)
    oos_pred.append(pred_Dense)  
    print(classification_report(y_test_Dense.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Dense.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

#   score = metrics.accuracy_score(y_test3_Incep.argmax(axis=1),	        #ค่า acc
#                                               pred3_Incep.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(testY3_Incep.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_Incep.argmax(axis=1) ) )

Fold #1
Epoch 1/30
45/45 [==============================] - 100s 2s/step - loss: 0.6230 - accuracy: 0.4250 - val_loss: 0.5992 - val_accuracy: 0.4505
Epoch 2/30
45/45 [==============================] - 141s 3s/step - loss: 0.6055 - accuracy: 0.4778 - val_loss: 0.5864 - val_accuracy: 0.4835
Epoch 3/30
45/45 [==============================] - 134s 3s/step - loss: 0.5867 - accuracy: 0.5583 - val_loss: 0.5721 - val_accuracy: 0.5824
Epoch 4/30
45/45 [==============================] - 154s 3s/step - loss: 0.5776 - accuracy: 0.5556 - val_loss: 0.5604 - val_accuracy: 0.6703
Epoch 5/30
45/45 [==============================] - 112s 3s/step - loss: 0.5652 - accuracy: 0.5861 - val_loss: 0.5502 - val_accuracy: 0.6374
Epoch 6/30
45/45 [==============================] - 113s 3s/step - loss: 0.5543 - accuracy: 0.6194 - val_loss: 0.5376 - val_accuracy: 0.7143
Epoch 7/30
45/45 [==============================] - 115s 3s/step - loss: 0.5425 - accuracy: 0.6444 - val_loss: 0.5308 - val_accuracy: 0.6484
Epoch

In [ ]:
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)

In [ ]:
score = metrics.accuracy_score(oos_y.argmax(axis=1),	        #ค่า acc
                                             oos_pred.argmax(axis=1) )
print(f"Fold score (accuracy): {score}")

print(classification_report(oos_y.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_pred.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

Fold score (accuracy): 0.6962305986696231
              precision    recall  f1-score   support

  Lungcancer       0.75      0.55      0.63       150
      Normal       0.67      0.97      0.79       151
Tuberculosis       0.69      0.57      0.63       150

    accuracy                           0.70       451
   macro avg       0.70      0.70      0.68       451
weighted avg       0.70      0.70      0.68       451



## 1.2) K-fold  -- DenseNet201

### -->ACC = 0.70 
### -->เวลา 4918.566 วินาที

*   BS = 8
*   activation="sigmoid"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))                                       #ขนาดของ input
    train_img_Dense.append(img)

data_Dense = np.array(train_img_Dense, dtype="float32") / 255.0

100%|██████████| 451/451 [00:05<00:00, 79.48it/s] 


In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
INIT_LR = 1e-1
BS = 8
EPOCHS = 50

aug = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
base_model_Dense.trainable=False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="sigmoid")(average_pooling_layer)   # unit 3
model_Dense = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
kf_Dense = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Dense.split(data_Dense):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train_Dense = data_Dense[train]
    y_train_Dense = labels_Dense[train]
    x_test_Dense = data_Dense[test]
    y_test_Dense = labels_Dense[test]

    H_Dense = model_Dense.fit( x=aug.flow(x_train_Dense, y_train_Dense, batch_size=BS),
	                                            validation_data=(x_test_Dense, y_test_Dense),
	                                            steps_per_epoch=len(x_train_Dense) // BS,
	                                            epochs=30,  callbacks = [callback])
    
    pred_Dense = model_Dense.predict(x_test_Dense)
    
    oos_y.append(y_test_Dense)
    oos_pred.append(pred_Dense)  
    print(classification_report(y_test_Dense.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Dense.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

#   score = metrics.accuracy_score(y_test3_Incep.argmax(axis=1),	        #ค่า acc
#                                               pred3_Incep.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(testY3_Incep.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_Incep.argmax(axis=1) ) )

Fold #1
Epoch 1/30
45/45 [==============================] - 152s 3s/step - loss: 0.6307 - accuracy: 0.4639 - val_loss: 0.5992 - val_accuracy: 0.5055
Epoch 2/30
45/45 [==============================] - 141s 3s/step - loss: 0.5935 - accuracy: 0.5333 - val_loss: 0.5830 - val_accuracy: 0.5275
Epoch 3/30
45/45 [==============================] - 105s 2s/step - loss: 0.5795 - accuracy: 0.5639 - val_loss: 0.5690 - val_accuracy: 0.5714
Epoch 4/30
45/45 [==============================] - 105s 2s/step - loss: 0.5651 - accuracy: 0.6083 - val_loss: 0.5597 - val_accuracy: 0.5714
Epoch 5/30
45/45 [==============================] - 105s 2s/step - loss: 0.5481 - accuracy: 0.6333 - val_loss: 0.5485 - val_accuracy: 0.5714
Epoch 6/30
45/45 [==============================] - 105s 2s/step - loss: 0.5472 - accuracy: 0.6222 - val_loss: 0.5421 - val_accuracy: 0.5714
              precision    recall  f1-score   support

  Lungcancer       0.55      0.39      0.46        28
      Normal       0.62      1.00    

In [ ]:
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)

In [ ]:
score = metrics.accuracy_score(oos_y.argmax(axis=1),	        #ค่า acc
                                             oos_pred.argmax(axis=1) )
print(f"Fold score (accuracy): {score}")

print(classification_report(oos_y.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_pred.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

Fold score (accuracy): 0.6962305986696231
              precision    recall  f1-score   support

  Lungcancer       0.77      0.61      0.68       150
      Normal       0.66      0.98      0.79       151
Tuberculosis       0.69      0.49      0.57       150

    accuracy                           0.70       451
   macro avg       0.71      0.70      0.68       451
weighted avg       0.71      0.70      0.68       451



## 2) K-fold  -- DenseNet201

### -->ACC = 0.65
### -->เวลา 34 นาที

*   BS = 16
*   activation="softmax"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))                                       #ขนาดของ input
    train_img_Dense.append(img)

data_Dense = np.array(train_img_Dense, dtype="float32") / 255.0

100%|██████████| 451/451 [00:04<00:00, 92.91it/s] 


In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
INIT_LR = 1e-1
BS16 = 16
EPOCHS = 50

aug2 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
base_model_Dense.trainable=False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Dense2 = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense2.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
kf_Dense2 = KFold(5, shuffle=True, random_state=40) 

oos_y2 = []
oos_pred2 = []
fold = 0

for train, test in kf_Dense2.split(data_Dense):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train_Dense2 = data_Dense[train]
    y_train_Dense2 = labels_Dense[train]
    x_test_Dense2 = data_Dense[test]
    y_test_Dense2 = labels_Dense[test]

    H_Dense2 = model_Dense2.fit( x=aug2.flow(x_train_Dense2, y_train_Dense2, batch_size=BS16),
	                                            validation_data=(x_test_Dense2, y_test_Dense2),
	                                            steps_per_epoch=len(x_train_Dense2) // BS16,
	                                            epochs=30,  callbacks = [callback])
    
    pred_Dense2 = model_Dense2.predict(x_test_Dense2)
    
    oos_y2.append(y_test_Dense2)
    oos_pred2.append(pred_Dense2)  
    print(classification_report(y_test_Dense2.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Dense2.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

#   score = metrics.accuracy_score(y_test3_Incep.argmax(axis=1),	        #ค่า acc
#                                               pred3_Incep.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(testY3_Incep.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_Incep.argmax(axis=1) ) )

Fold #1
Epoch 1/30
22/22 [==============================] - 109s 4s/step - loss: 0.6742 - accuracy: 0.3837 - val_loss: 0.6225 - val_accuracy: 0.5495
Epoch 2/30
22/22 [==============================] - 98s 5s/step - loss: 0.6248 - accuracy: 0.4506 - val_loss: 0.6098 - val_accuracy: 0.5714
Epoch 3/30
22/22 [==============================] - 98s 4s/step - loss: 0.6055 - accuracy: 0.4971 - val_loss: 0.5992 - val_accuracy: 0.5824
Epoch 4/30
22/22 [==============================] - 103s 5s/step - loss: 0.6024 - accuracy: 0.5436 - val_loss: 0.5895 - val_accuracy: 0.6264
Epoch 5/30
22/22 [==============================] - 100s 5s/step - loss: 0.5971 - accuracy: 0.5116 - val_loss: 0.5807 - val_accuracy: 0.6374
Epoch 6/30
22/22 [==============================] - 99s 5s/step - loss: 0.5824 - accuracy: 0.5785 - val_loss: 0.5746 - val_accuracy: 0.6154
Epoch 7/30
22/22 [==============================] - 100s 5s/step - loss: 0.5668 - accuracy: 0.6192 - val_loss: 0.5659 - val_accuracy: 0.6593
Epoch 8/

In [ ]:
oos_y2 = np.concatenate(oos_y2)
oos_pred2 = np.concatenate(oos_pred2)

In [ ]:
score = metrics.accuracy_score(oos_y2.argmax(axis=1),	        #ค่า acc
                                             oos_pred2.argmax(axis=1) )
print(f"Fold score (accuracy): {score}")

print(classification_report(oos_y2.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_pred2.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

Fold score (accuracy): 0.6541019955654102
              precision    recall  f1-score   support

  Lungcancer       0.80      0.52      0.63       150
      Normal       0.61      0.97      0.75       151
Tuberculosis       0.63      0.47      0.54       150

    accuracy                           0.65       451
   macro avg       0.68      0.65      0.64       451
weighted avg       0.68      0.65      0.64       451



## 2.1) K-fold  -- DenseNet201

### -->ACC = 0.69
### -->เวลา 5093.751 นาที

*   BS = 16
*   activation="sigmoid"
*   learning_rate=0.0001
*   epoch 30

In [13]:
train_img_Dense = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))                                       #ขนาดของ input
    train_img_Dense.append(img)

data_Dense = np.array(train_img_Dense, dtype="float32") / 255.0

100%|██████████| 451/451 [02:08<00:00,  3.51it/s]


In [14]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [15]:
INIT_LR = 1e-1
BS16 = 16
EPOCHS = 50

aug21 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [16]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
base_model_Dense.trainable=False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="sigmoid")(average_pooling_layer)   # unit 3
model_Dense21 = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense21.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

74850304/74836368 [==============================] - 2s 0us/step


In [17]:
kf_Dense21 = KFold(5, shuffle=True, random_state=40) 

oos_y21 = []
oos_pred21 = []
fold = 0

for train, test in kf_Dense21.split(data_Dense):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train_Dense21 = data_Dense[train]
    y_train_Dense21 = labels_Dense[train]
    x_test_Dense21 = data_Dense[test]
    y_test_Dense21 = labels_Dense[test]

    H_Dense21 = model_Dense21.fit( x=aug21.flow(x_train_Dense21, y_train_Dense21, batch_size=BS16),
	                                            validation_data=(x_test_Dense21, y_test_Dense21),
	                                            steps_per_epoch=len(x_train_Dense21) // BS16,
	                                            epochs=30,  callbacks = [callback])
    
    pred_Dense21 = model_Dense21.predict(x_test_Dense21)
    
    oos_y21.append(y_test_Dense21)
    oos_pred21.append(pred_Dense21)  
    print(classification_report(y_test_Dense21.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Dense21.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

#   score = metrics.accuracy_score(y_test3_Incep.argmax(axis=1),	        #ค่า acc
#                                               pred3_Incep.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(testY3_Incep.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_Incep.argmax(axis=1) ) )

Fold #1
Epoch 1/30
22/22 [==============================] - 104s 4s/step - loss: 0.6511 - accuracy: 0.4157 - val_loss: 0.6298 - val_accuracy: 0.4066
Epoch 2/30
22/22 [==============================] - 89s 4s/step - loss: 0.6400 - accuracy: 0.3866 - val_loss: 0.6223 - val_accuracy: 0.4396
Epoch 3/30
22/22 [==============================] - 89s 4s/step - loss: 0.6278 - accuracy: 0.4273 - val_loss: 0.6128 - val_accuracy: 0.5055
Epoch 4/30
22/22 [==============================] - 88s 4s/step - loss: 0.6184 - accuracy: 0.4651 - val_loss: 0.6035 - val_accuracy: 0.5385
Epoch 5/30
22/22 [==============================] - 89s 4s/step - loss: 0.6068 - accuracy: 0.5087 - val_loss: 0.5941 - val_accuracy: 0.5824
Epoch 6/30
22/22 [==============================] - 88s 4s/step - loss: 0.5986 - accuracy: 0.5233 - val_loss: 0.5856 - val_accuracy: 0.6154
Epoch 7/30
22/22 [==============================] - 89s 4s/step - loss: 0.5862 - accuracy: 0.5552 - val_loss: 0.5780 - val_accuracy: 0.6703
Epoch 8/30


In [18]:
oos_y21 = np.concatenate(oos_y21)
oos_pred21 = np.concatenate(oos_pred21)

In [19]:
score = metrics.accuracy_score(oos_y21.argmax(axis=1),	        #ค่า acc
                                             oos_pred21.argmax(axis=1) )
print(f"Fold score (accuracy): {score}")

print(classification_report(oos_y21.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_pred21.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

Fold score (accuracy): 0.6940133037694013
              precision    recall  f1-score   support

  Lungcancer       0.75      0.65      0.70       150
      Normal       0.64      0.99      0.78       151
Tuberculosis       0.74      0.44      0.55       150

    accuracy                           0.69       451
   macro avg       0.71      0.69      0.68       451
weighted avg       0.71      0.69      0.68       451



## 3) K-fold  -- DenseNet201

### -->ACC = 0.65 
### -->เวลา 5419.91 วินาที

*   BS = 32
*   activation="softmax"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))                                       #ขนาดของ input
    train_img_Dense.append(img)

data_Dense = np.array(train_img_Dense, dtype="float32") / 255.0

100%|██████████| 451/451 [00:05<00:00, 83.88it/s] 


In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
INIT_LR = 1e-1
BS32 = 32
EPOCHS = 50

aug3 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
base_model_Dense.trainable=False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Dense3 = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense3.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
kf_Dense3 = KFold(5, shuffle=True, random_state=40) 

oos_y3 = []
oos_pred3 = []
fold = 0

for train, test in kf_Dense3.split(data_Dense):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train_Dense3 = data_Dense[train]
    y_train_Dense3 = labels_Dense[train]
    x_test_Dense3 = data_Dense[test]
    y_test_Dense3 = labels_Dense[test]

    H_Dense3 = model_Dense3.fit( x=aug3.flow(x_train_Dense3, y_train_Dense3, batch_size=BS32),
	                                            validation_data=(x_test_Dense3, y_test_Dense3),
	                                            steps_per_epoch=len(x_train_Dense3) // BS32,
	                                            epochs=30,  callbacks = [callback])
    
    pred_Dense3 = model_Dense3.predict(x_test_Dense3)
    
    oos_y3.append(y_test_Dense3)
    oos_pred3.append(pred_Dense3)  
    print(classification_report(y_test_Dense3.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Dense3.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

#   score = metrics.accuracy_score(y_test3_Incep.argmax(axis=1),	        #ค่า acc
#                                               pred3_Incep.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(testY3_Incep.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_Incep.argmax(axis=1) ) )

Fold #1
Epoch 1/30
11/11 [==============================] - 97s 8s/step - loss: 0.7570 - accuracy: 0.2896 - val_loss: 0.6820 - val_accuracy: 0.3407
Epoch 2/30
11/11 [==============================] - 82s 8s/step - loss: 0.6755 - accuracy: 0.3293 - val_loss: 0.6577 - val_accuracy: 0.3846
Epoch 3/30
11/11 [==============================] - 82s 8s/step - loss: 0.6624 - accuracy: 0.3110 - val_loss: 0.6504 - val_accuracy: 0.3626
Epoch 4/30
11/11 [==============================] - 82s 8s/step - loss: 0.6525 - accuracy: 0.3537 - val_loss: 0.6447 - val_accuracy: 0.3516
Epoch 5/30
11/11 [==============================] - 82s 8s/step - loss: 0.6519 - accuracy: 0.3445 - val_loss: 0.6387 - val_accuracy: 0.3956
Epoch 6/30
11/11 [==============================] - 86s 8s/step - loss: 0.6336 - accuracy: 0.4119 - val_loss: 0.6321 - val_accuracy: 0.3956
Epoch 7/30
11/11 [==============================] - 82s 8s/step - loss: 0.6294 - accuracy: 0.3750 - val_loss: 0.6255 - val_accuracy: 0.4505
Epoch 8/30
1

In [ ]:
oos_y3 = np.concatenate(oos_y3)
oos_pred3 = np.concatenate(oos_pred3)

In [ ]:
score = metrics.accuracy_score(oos_y3.argmax(axis=1),	        #ค่า acc
                                             oos_pred3.argmax(axis=1) )
print(f"Fold score (accuracy): {score}")

print(classification_report(oos_y3.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_pred3.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

Fold score (accuracy): 0.6541019955654102
              precision    recall  f1-score   support

  Lungcancer       0.80      0.48      0.60       150
      Normal       0.63      0.87      0.73       151
Tuberculosis       0.60      0.61      0.61       150

    accuracy                           0.65       451
   macro avg       0.68      0.65      0.65       451
weighted avg       0.68      0.65      0.65       451



## 3.1) K-fold  -- DenseNet201  ก็อปไปใส่ไฟล์ DenseNet201_pretrained_bs แล้ว

### -->ACC = 0.65 
### -->เวลา 5985.206 วินาที

*   BS = 32
*   activation="sigmoid"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))                                       #ขนาดของ input
    train_img_Dense.append(img)

data_Dense = np.array(train_img_Dense, dtype="float32") / 255.0

100%|██████████| 451/451 [00:04<00:00, 93.45it/s] 


In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
INIT_LR = 1e-1
BS32 = 32
EPOCHS = 50

aug31 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
base_model_Dense.trainable=False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="sigmoid")(average_pooling_layer)   # unit 3
model_Dense31 = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense31.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
kf_Dense31 = KFold(5, shuffle=True, random_state=40) 

oos_y31 = []
oos_pred31 = []
fold = 0

for train, test in kf_Dense31.split(data_Dense):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train_Dense31 = data_Dense[train]
    y_train_Dense31 = labels_Dense[train]
    x_test_Dense31 = data_Dense[test]
    y_test_Dense31 = labels_Dense[test]

    H_Dense31 = model_Dense31.fit( x=aug31.flow(x_train_Dense31, y_train_Dense31, batch_size=BS32),
	                                            validation_data=(x_test_Dense31, y_test_Dense31),
	                                            steps_per_epoch=len(x_train_Dense31) // BS32,
	                                            epochs=30,  callbacks = [callback])
    
    pred_Dense31 = model_Dense31.predict(x_test_Dense31)
    
    oos_y31.append(y_test_Dense31)
    oos_pred31.append(pred_Dense31)  
    print(classification_report(y_test_Dense31.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Dense31.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

#   score = metrics.accuracy_score(y_test3_Incep.argmax(axis=1),	        #ค่า acc
#                                               pred3_Incep.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(testY3_Incep.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_Incep.argmax(axis=1) ) )

Fold #1
Epoch 1/30
11/11 [==============================] - 99s 8s/step - loss: 0.7409 - accuracy: 0.3018 - val_loss: 0.7162 - val_accuracy: 0.2527
Epoch 2/30
11/11 [==============================] - 86s 8s/step - loss: 0.6801 - accuracy: 0.2988 - val_loss: 0.6940 - val_accuracy: 0.2747
Epoch 3/30
11/11 [==============================] - 85s 8s/step - loss: 0.6637 - accuracy: 0.2805 - val_loss: 0.6833 - val_accuracy: 0.2967
Epoch 4/30
11/11 [==============================] - 85s 8s/step - loss: 0.6521 - accuracy: 0.3262 - val_loss: 0.6761 - val_accuracy: 0.3187
Epoch 5/30
11/11 [==============================] - 85s 8s/step - loss: 0.6464 - accuracy: 0.3354 - val_loss: 0.6698 - val_accuracy: 0.3297
Epoch 6/30
11/11 [==============================] - 89s 9s/step - loss: 0.6323 - accuracy: 0.4299 - val_loss: 0.6630 - val_accuracy: 0.3297
Epoch 7/30
11/11 [==============================] - 88s 8s/step - loss: 0.6264 - accuracy: 0.4360 - val_loss: 0.6552 - val_accuracy: 0.4176
Epoch 8/30
1

In [ ]:
oos_y31 = np.concatenate(oos_y31)
oos_pred31 = np.concatenate(oos_pred31)

In [ ]:
score = metrics.accuracy_score(oos_y31.argmax(axis=1),	        #ค่า acc
                                             oos_pred31.argmax(axis=1) )
print(f"Fold score (accuracy): {score}")

print(classification_report(oos_y31.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_pred31.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

Fold score (accuracy): 0.647450110864745
              precision    recall  f1-score   support

  Lungcancer       0.75      0.52      0.61       150
      Normal       0.62      0.98      0.76       151
Tuberculosis       0.62      0.44      0.51       150

    accuracy                           0.65       451
   macro avg       0.66      0.65      0.63       451
weighted avg       0.66      0.65      0.63       451



## 4) K-fold  -- DenseNet201 

### -->ACC = 0.72
### -->เวลา 4859 วินาที

*   BS = 1
*   activation="softmax"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))                                       #ขนาดของ input
    train_img_Dense.append(img)

data_Dense = np.array(train_img_Dense, dtype="float32") / 255.0

100%|██████████| 451/451 [02:08<00:00,  3.51it/s]


In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
INIT_LR = 1e-1
BS = 1
EPOCHS = 50

aug4 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
base_model_Dense.trainable=False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Dense4 = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense4.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

74850304/74836368 [==============================] - 1s 0us/step


In [ ]:
kf_Dense4 = KFold(5, shuffle=True, random_state=40) 

oos_y4 = []
oos_pred4 = []
fold = 0

for train, test in kf_Dense4.split(data_Dense):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train_Dense4 = data_Dense[train]
    y_train_Dense4 = labels_Dense[train]
    x_test_Dense4 = data_Dense[test]
    y_test_Dense4 = labels_Dense[test]

    H_Dense4 = model_Dense4.fit( x=aug4.flow(x_train_Dense4, y_train_Dense4, batch_size=BS),
	                                            validation_data=(x_test_Dense4, y_test_Dense4),
	                                            steps_per_epoch=len(x_train_Dense4) // BS,
	                                            epochs=30,  callbacks = [callback])
    
    pred_Dense4 = model_Dense4.predict(x_test_Dense4)
    
    oos_y4.append(y_test_Dense4)
    oos_pred4.append(pred_Dense4)  
    print(classification_report(y_test_Dense4.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Dense4.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

#   score = metrics.accuracy_score(y_test3_Incep.argmax(axis=1),	        #ค่า acc
#                                               pred3_Incep.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(testY3_Incep.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_Incep.argmax(axis=1) ) )

Fold #1
Epoch 1/30
360/360 [==============================] - 119s 301ms/step - loss: 0.6247 - accuracy: 0.4861 - val_loss: 0.5918 - val_accuracy: 0.6374
Epoch 2/30
360/360 [==============================] - 113s 313ms/step - loss: 0.5784 - accuracy: 0.5861 - val_loss: 0.5540 - val_accuracy: 0.6264
Epoch 3/30
360/360 [==============================] - 171s 474ms/step - loss: 0.5459 - accuracy: 0.6139 - val_loss: 0.5396 - val_accuracy: 0.6264
Epoch 4/30
360/360 [==============================] - 192s 534ms/step - loss: 0.5245 - accuracy: 0.6639 - val_loss: 0.5129 - val_accuracy: 0.6264
              precision    recall  f1-score   support

  Lungcancer       0.58      0.68      0.62        28
      Normal       0.65      0.97      0.78        33
Tuberculosis       0.67      0.20      0.31        30

    accuracy                           0.63        91
   macro avg       0.63      0.62      0.57        91
weighted avg       0.63      0.63      0.58        91

Fold #2
Epoch 1/30
361/361 

In [ ]:
oos_y4 = np.concatenate(oos_y4)
oos_pred4 = np.concatenate(oos_pred4)

In [ ]:
score = metrics.accuracy_score(oos_y4.argmax(axis=1),	        #ค่า acc
                                             oos_pred4.argmax(axis=1) )
print(f"Fold score (accuracy): {score}")

print(classification_report(oos_y4.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_pred4.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

Fold score (accuracy): 0.7161862527716186
              precision    recall  f1-score   support

  Lungcancer       0.71      0.73      0.72       150
      Normal       0.69      0.98      0.81       151
Tuberculosis       0.78      0.44      0.56       150

    accuracy                           0.72       451
   macro avg       0.73      0.72      0.70       451
weighted avg       0.73      0.72      0.70       451



## 4.1) K-fold  -- DenseNet201  

### -->ACC = 0.71
### -->เวลา 3232.937 วินาที หรือ 53 นาที

*   BS = 1
*   activation="sigmoid"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))                                       #ขนาดของ input
    train_img_Dense.append(img)

data_Dense = np.array(train_img_Dense, dtype="float32") / 255.0

100%|██████████| 451/451 [00:04<00:00, 94.46it/s] 


In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
INIT_LR = 1e-1
BS = 1
EPOCHS = 50

aug41 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
base_model_Dense.trainable=False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="sigmoid")(average_pooling_layer)   # unit 3
model_Dense41 = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense41.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
kf_Dense41 = KFold(5, shuffle=True, random_state=40) 

oos_y41 = []
oos_pred41 = []
fold = 0

for train, test in kf_Dense41.split(data_Dense):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train_Dense41 = data_Dense[train]
    y_train_Dense41 = labels_Dense[train]
    x_test_Dense41 = data_Dense[test]
    y_test_Dense41 = labels_Dense[test]

    H_Dense41 = model_Dense41.fit( x=aug41.flow(x_train_Dense41, y_train_Dense41, batch_size=BS),
	                                            validation_data=(x_test_Dense41, y_test_Dense41),
	                                            steps_per_epoch=len(x_train_Dense41) // BS,
	                                            epochs=30,  callbacks = [callback])
    
    pred_Dense41 = model_Dense41.predict(x_test_Dense41)
    
    oos_y41.append(y_test_Dense41)
    oos_pred41.append(pred_Dense41)  
    print(classification_report(y_test_Dense41.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Dense41.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

#   score = metrics.accuracy_score(y_test3_Incep.argmax(axis=1),	        #ค่า acc
#                                               pred3_Incep.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(testY3_Incep.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_Incep.argmax(axis=1) ) )

Fold #1
Epoch 1/30
360/360 [==============================] - 124s 317ms/step - loss: 0.6712 - accuracy: 0.3111 - val_loss: 0.6311 - val_accuracy: 0.4176
Epoch 2/30
360/360 [==============================] - 136s 377ms/step - loss: 0.6247 - accuracy: 0.4306 - val_loss: 0.5933 - val_accuracy: 0.5385
Epoch 3/30
360/360 [==============================] - 137s 381ms/step - loss: 0.5871 - accuracy: 0.5389 - val_loss: 0.5626 - val_accuracy: 0.6484
Epoch 4/30
360/360 [==============================] - 125s 348ms/step - loss: 0.5525 - accuracy: 0.5917 - val_loss: 0.5368 - val_accuracy: 0.6484
Epoch 5/30
360/360 [==============================] - 114s 317ms/step - loss: 0.5319 - accuracy: 0.6472 - val_loss: 0.5142 - val_accuracy: 0.6484
Epoch 6/30
360/360 [==============================] - 118s 329ms/step - loss: 0.5138 - accuracy: 0.6722 - val_loss: 0.4983 - val_accuracy: 0.6484
              precision    recall  f1-score   support

  Lungcancer       0.59      0.36      0.44        28
      N

In [ ]:
oos_y41 = np.concatenate(oos_y41)
oos_pred41 = np.concatenate(oos_pred41)

In [ ]:
score = metrics.accuracy_score(oos_y41.argmax(axis=1),	        #ค่า acc
                                             oos_pred41.argmax(axis=1) )
print(f"Fold score (accuracy): {score}")

print(classification_report(oos_y41.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_pred41.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

Fold score (accuracy): 0.70509977827051
              precision    recall  f1-score   support

  Lungcancer       0.75      0.59      0.66       150
      Normal       0.70      0.98      0.82       151
Tuberculosis       0.68      0.54      0.60       150

    accuracy                           0.71       451
   macro avg       0.71      0.70      0.69       451
weighted avg       0.71      0.71      0.69       451



# ส่วนใหม่

## 1) Find Tuning

### -->ACC = 0.90

*   BS = 32
*   activation="softmax"
*   fix weight & bias in layer 0-100
*   learning_rate=0.0001
*   epoch 15

##### Data

In [ ]:
train_img3_Incep = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (299, 299))                                       #ขนาดของ input
    train_img3_Incep.append(img)

data3_Incep = np.array(train_img3_Incep, dtype="float32") / 255.0

100%|██████████| 451/451 [01:58<00:00,  3.80it/s]


In [ ]:
labels3_Incep = np.array(data_train3[['NameType']] )        
labels3_Incep.shape = (len(labels3_Incep),) 

le3_Incep = LabelEncoder()                                                                     
labels3_Incep = le3_Incep.fit_transform(labels3_Incep)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels3_Incep = to_categorical(labels3_Incep, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

##### Model

In [ ]:
base_model3_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model3_Incep.layers)

87924736/87910968 [==============================] - 0s 0us/step


311

In [ ]:
base_model3_Incep.trainable = True
for layer in base_model3_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False

In [ ]:
#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model3_Incep.output)

prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model3_IncepF = tf.keras.models.Model(inputs=base_model3_Incep.input, outputs=prediction_layer)

model3_IncepF.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
INIT_LR = 1e-1
BS = 32
EPOCHS = 50

aug = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
kf_IncepF = KFold(5, shuffle=True, random_state=40) 

oos_yF = []
oos_predF = []
fold = 0

for train, test in kf_IncepF.split(data3_Incep):
    fold+=1
    print(f"Fold #{fold}")
        
    xtrain_IncepF = data3_Incep[train]
    ytrain_IncepF = labels3_Incep[train]
    xtest_IncepF = data3_Incep[test]
    ytest_IncepF = labels3_Incep[test]

    M_IncepF = model3_IncepF.fit( x=aug.flow(xtrain_IncepF, ytrain_IncepF, batch_size=BS),
	                                            validation_data=(xtest_IncepF, ytest_IncepF),
	                                            steps_per_epoch=len(xtrain_IncepF) // BS,
	                                            epochs=15,  callbacks = [callback])
    
    pred_IncepF = model3_IncepF.predict(xtest_IncepF)
    
    oos_yF.append(ytest_IncepF)
    oos_predF.append(pred_IncepF)  
    print(classification_report(ytest_IncepF.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_IncepF.argmax(axis=1),  
                                        target_names=le3_Incep.classes_))

#   score = metrics.accuracy_score(ytest_IncepF.argmax(axis=1),	        #ค่า acc
#                                               pred_IncepF.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(ytest_IncepF.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_IncepF.argmax(axis=1) ) )

Fold #1
Epoch 1/15
11/11 [==============================] - 212s 18s/step - loss: 0.5453 - accuracy: 0.5701 - val_loss: 0.8733 - val_accuracy: 0.4396
Epoch 2/15
11/11 [==============================] - 188s 17s/step - loss: 0.3283 - accuracy: 0.7957 - val_loss: 0.8059 - val_accuracy: 0.4725
Epoch 3/15
11/11 [==============================] - 188s 17s/step - loss: 0.2771 - accuracy: 0.8476 - val_loss: 0.7430 - val_accuracy: 0.5604
Epoch 4/15
11/11 [==============================] - 188s 17s/step - loss: 0.1564 - accuracy: 0.9238 - val_loss: 0.7924 - val_accuracy: 0.5385
Epoch 5/15
11/11 [==============================] - 188s 17s/step - loss: 0.1670 - accuracy: 0.9207 - val_loss: 0.8088 - val_accuracy: 0.5714
Epoch 6/15
11/11 [==============================] - 188s 17s/step - loss: 0.1128 - accuracy: 0.9421 - val_loss: 0.8733 - val_accuracy: 0.5714
Epoch 7/15
11/11 [==============================] - 188s 18s/step - loss: 0.1176 - accuracy: 0.9299 - val_loss: 1.0824 - val_accuracy: 0.527

In [ ]:
oos_yF = np.concatenate(oos_yF)
oos_predF = np.concatenate(oos_predF)

In [ ]:
score = metrics.accuracy_score(oos_yF.argmax(axis=1),	        #ค่า acc
                                             oos_predF.argmax(axis=1) )
print(f"Fold score (accuracy): {score}")

print(classification_report(oos_yF.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_predF.argmax(axis=1),  
                                        target_names=le3_Incep.classes_))

Fold score (accuracy): 0.9046563192904656
              precision    recall  f1-score   support

  Lungcancer       1.00      0.82      0.90       150
      Normal       0.99      0.90      0.94       151
Tuberculosis       0.78      0.99      0.88       150

    accuracy                           0.90       451
   macro avg       0.92      0.90      0.91       451
weighted avg       0.92      0.90      0.91       451



In [ ]:
#ลองเซฟโมเดล
fold = 2
if metrics.accuracy_score( ytest_IncepF.argmax(axis=1),  pred_IncepF.argmax(axis=1) )>0.7:
    name = (f' model3_IncepF{fold}')
    joblib.dump(model3_IncepF, name)

INFO:tensorflow:Assets written to: ram://a5d1d51e-3da4-454e-81fb-752e887bd981/assets


## 2) Find Tuning  

###--> ACC = 0.76  
###--> ใช้เวลา 8173.251 วินาที = 136.22 นาที = 2.27 ชั่วโมง

*   BS = 16
*   activation="softmax"
*   fix weight & bias in layer 0-100
*   learning_rate=0.0001
*   epoch 30


##### Data

In [ ]:
train_img3_Incep = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (299, 299))                                       #ขนาดของ input
    train_img3_Incep.append(img)

data3_Incep = np.array(train_img3_Incep, dtype="float32") / 255.0

100%|██████████| 451/451 [03:31<00:00,  2.13it/s]


In [ ]:
labels3_Incep = np.array(data_train3[['NameType']] )        
labels3_Incep.shape = (len(labels3_Incep),) 

le3_Incep = LabelEncoder()                                                                     
labels3_Incep = le3_Incep.fit_transform(labels3_Incep)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels3_Incep = to_categorical(labels3_Incep, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

##### Model

In [ ]:
base_model3_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model3_Incep.layers)

87924736/87910968 [==============================] - 1s 0us/step


311

In [ ]:
base_model3_Incep.trainable = True
for layer in base_model3_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False

In [ ]:
#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model3_Incep.output)

prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model3_IncepF2 = tf.keras.models.Model(inputs=base_model3_Incep.input, outputs=prediction_layer)

model3_IncepF2.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
INIT_LR = 1e-1
BS = 16
EPOCHS = 50

aug2 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.15,				  #เลื่อนลง-ขึ้น ซ้ายขวา
		height_shift_range=0.15,
		shear_range=0.15, 						#ขยับมุมภาพ ซ้าย และ/หรือ ขวา ไปด้านข้าง เช่น จะกลายเป็นสี่เหลี่ยมด้านขนาน 
		horizontal_flip=True,
		fill_mode='constant')

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
kf_IncepF2 = KFold(5, shuffle=True, random_state=40) 

oos_yF2 = []
oos_predF2 = []
fold = 0

for train, test in kf_IncepF2.split(data3_Incep):
    fold+=1
    print(f"Fold #{fold}")
        
    xtrain_IncepF2 = data3_Incep[train]
    ytrain_IncepF2 = labels3_Incep[train]
    xtest_IncepF2 = data3_Incep[test]
    ytest_IncepF2 = labels3_Incep[test]

    M_IncepF2 = model3_IncepF2.fit( x=aug2.flow(xtrain_IncepF2, ytrain_IncepF2, batch_size=BS),
	                                            validation_data=(xtest_IncepF2, ytest_IncepF2),
	                                            steps_per_epoch=len(xtrain_IncepF2) // BS,
	                                            epochs=30,  callbacks = [callback])
    
    pred_IncepF2 = model3_IncepF2.predict(xtest_IncepF2)

    oos_yF2.append(ytest_IncepF2)
    oos_predF2.append(pred_IncepF2) 

    score = metrics.accuracy_score(ytest_IncepF2.argmax(axis=1),	        #ค่า acc
                                               pred_IncepF2.argmax(axis=1) )
    print(f'Fold #{fold} score (accuracy): {score}')
    

#    if metrics.accuracy_score( ytest_IncepF2.argmax(axis=1),  pred_IncepF2.argmax(axis=1) )>0.9:
 #       name = (f' model3_IncepF2_Fold{fold}')
  #      joblib.dump(model3_IncepF2, name)

#    print(confusion_matrix(ytest_IncepF.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_IncepF.argmax(axis=1) ) )

Fold #1
Epoch 1/30
22/22 [==============================] - 187s 8s/step - loss: 0.4612 - accuracy: 0.6686 - val_loss: 1.4591 - val_accuracy: 0.3736
Epoch 2/30
22/22 [==============================] - 175s 8s/step - loss: 0.3046 - accuracy: 0.8052 - val_loss: 1.3021 - val_accuracy: 0.4396
Epoch 3/30
22/22 [==============================] - 173s 8s/step - loss: 0.2600 - accuracy: 0.8314 - val_loss: 1.0278 - val_accuracy: 0.5055
Epoch 4/30
22/22 [==============================] - 174s 8s/step - loss: 0.2032 - accuracy: 0.8634 - val_loss: 1.0180 - val_accuracy: 0.5714
Epoch 5/30
22/22 [==============================] - 174s 8s/step - loss: 0.1327 - accuracy: 0.9331 - val_loss: 1.1071 - val_accuracy: 0.6044
Epoch 6/30
22/22 [==============================] - 174s 8s/step - loss: 0.1238 - accuracy: 0.9331 - val_loss: 0.7453 - val_accuracy: 0.6813
Epoch 7/30
22/22 [==============================] - 174s 8s/step - loss: 0.1349 - accuracy: 0.9390 - val_loss: 0.4770 - val_accuracy: 0.7692
Epoch

In [ ]:
oos_yF2 = np.concatenate(oos_yF2)
oos_predF2 = np.concatenate(oos_predF2)

In [ ]:
print(classification_report(oos_yF2.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_predF2.argmax(axis=1),  
                                        target_names=le3_Incep.classes_))

              precision    recall  f1-score   support

  Lungcancer       0.99      0.59      0.74       150
      Normal       0.93      0.72      0.81       151
Tuberculosis       0.60      0.98      0.74       150

    accuracy                           0.76       451
   macro avg       0.84      0.76      0.76       451
weighted avg       0.84      0.76      0.76       451



## 3) Find Tuning 
###--> ACC = 81
###--> 5815.668 วินาที

*   BS = 8
*   activation="softmax"
*   fix weight & bias in layer 0-100
*   learning_rate=0.0001
*   epoch 30







In [ ]:
train_img3_Incep = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (299, 299))                                       #ขนาดของ input
    train_img3_Incep.append(img)

data3_Incep = np.array(train_img3_Incep, dtype="float32") / 255.0

100%|██████████| 451/451 [00:05<00:00, 83.92it/s] 


In [ ]:
labels3_Incep = np.array(data_train3[['NameType']] )        
labels3_Incep.shape = (len(labels3_Incep),) 

le3_Incep = LabelEncoder()                                                                     
labels3_Incep = le3_Incep.fit_transform(labels3_Incep)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels3_Incep = to_categorical(labels3_Incep, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

##### Model

In [ ]:
base_model3_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model3_Incep.layers)

311

In [ ]:
base_model3_Incep.trainable = True
for layer in base_model3_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False

In [ ]:
#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model3_Incep.output)

prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model3_IncepF3 = tf.keras.models.Model(inputs=base_model3_Incep.input, outputs=prediction_layer)

model3_IncepF3.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
INIT_LR = 1e-1
BS = 8

aug3 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
kf_IncepF3 = KFold(5, shuffle=True, random_state=40) 

oos_yF3 = []
oos_predF3 = []
fold = 0

for train, test in kf_IncepF3.split(data3_Incep):
    fold+=1
    print(f"Fold #{fold}")
        
    xtrain_IncepF3 = data3_Incep[train]
    ytrain_IncepF3 = labels3_Incep[train]
    xtest_IncepF3 = data3_Incep[test]
    ytest_IncepF3 = labels3_Incep[test]

    M_IncepF3 = model3_IncepF3.fit( x=aug3.flow(xtrain_IncepF3, ytrain_IncepF3, batch_size=BS),
	                                            validation_data=(xtest_IncepF3, ytest_IncepF3),
	                                            steps_per_epoch=len(xtrain_IncepF3) // BS,
	                                            epochs=30,  callbacks = [callback])
    
    pred_IncepF3 = model3_IncepF3.predict(xtest_IncepF3)
    
    oos_yF3.append(ytest_IncepF3)
    oos_predF3.append(pred_IncepF3)  
    print(classification_report(ytest_IncepF3.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_IncepF3.argmax(axis=1),  
                                        target_names=le3_Incep.classes_))

#   score = metrics.accuracy_score(ytest_IncepF.argmax(axis=1),	        #ค่า acc
#                                               pred_IncepF.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(ytest_IncepF.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_IncepF.argmax(axis=1) ) )

Fold #1
Epoch 1/30
45/45 [==============================] - 195s 4s/step - loss: 0.4942 - accuracy: 0.6361 - val_loss: 0.7942 - val_accuracy: 0.4286
Epoch 2/30
45/45 [==============================] - 183s 4s/step - loss: 0.3243 - accuracy: 0.8000 - val_loss: 1.0859 - val_accuracy: 0.4835
Epoch 3/30
45/45 [==============================] - 183s 4s/step - loss: 0.2680 - accuracy: 0.8333 - val_loss: 1.0556 - val_accuracy: 0.5275
Epoch 4/30
45/45 [==============================] - 183s 4s/step - loss: 0.2613 - accuracy: 0.8694 - val_loss: 0.3897 - val_accuracy: 0.8022
Epoch 5/30
45/45 [==============================] - 183s 4s/step - loss: 0.2072 - accuracy: 0.8944 - val_loss: 0.8960 - val_accuracy: 0.6044
Epoch 6/30
45/45 [==============================] - 183s 4s/step - loss: 0.1478 - accuracy: 0.9083 - val_loss: 0.5644 - val_accuracy: 0.7692
Epoch 7/30
45/45 [==============================] - 188s 4s/step - loss: 0.1602 - accuracy: 0.9250 - val_loss: 1.2475 - val_accuracy: 0.6813
     

In [ ]:
oos_yF3 = np.concatenate(oos_yF3)
oos_predF3 = np.concatenate(oos_predF3)

In [ ]:
print(classification_report(oos_yF3.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_predF3.argmax(axis=1),  
                                        target_names=le3_Incep.classes_))

              precision    recall  f1-score   support

  Lungcancer       1.00      0.54      0.70       150
      Normal       0.94      0.91      0.93       151
Tuberculosis       0.66      0.98      0.79       150

    accuracy                           0.81       451
   macro avg       0.87      0.81      0.81       451
weighted avg       0.87      0.81      0.81       451



## 3.1) Find Tuning  

###--> ACC = 0.72
###--> ใช้เวลา 6354.25  วินาที 

*   BS = 8
*   activation="sigmoid"
*   fix weight & bias in layer 0-100
*   learning_rate=0.0001
*   epoch 30


##### Data

In [ ]:
train_img3_Incep = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (299, 299))                                       #ขนาดของ input
    train_img3_Incep.append(img)

data3_Incep = np.array(train_img3_Incep, dtype="float32") / 255.0

100%|██████████| 451/451 [00:05<00:00, 86.56it/s] 


In [ ]:
labels3_Incep = np.array(data_train3[['NameType']] )        
labels3_Incep.shape = (len(labels3_Incep),) 

le3_Incep = LabelEncoder()                                                                     
labels3_Incep = le3_Incep.fit_transform(labels3_Incep)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels3_Incep = to_categorical(labels3_Incep, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

##### Model

In [ ]:
base_model3_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model3_Incep.layers)

311

In [ ]:
base_model3_Incep.trainable = True
for layer in base_model3_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False

In [ ]:
#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model3_Incep.output)

prediction_layer = tf.keras.layers.Dense(units=3, activation="sigmoid")(average_pooling_layer)   # unit 3
model3_IncepF31 = tf.keras.models.Model(inputs=base_model3_Incep.input, outputs=prediction_layer)

model3_IncepF31.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
INIT_LR = 1e-1
BS = 8

aug31 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
kf_IncepF31 = KFold(5, shuffle=True, random_state=40) 

oos_yF31 = []
oos_predF31 = []
fold = 0

for train, test in kf_IncepF31.split(data3_Incep):
    fold+=1
    print(f"Fold #{fold}")
        
    xtrain_IncepF31 = data3_Incep[train]
    ytrain_IncepF31 = labels3_Incep[train]
    xtest_IncepF31 = data3_Incep[test]
    ytest_IncepF31 = labels3_Incep[test]

    M_IncepF31 = model3_IncepF31.fit( x=aug31.flow(xtrain_IncepF31, ytrain_IncepF31, batch_size=BS),
	                                            validation_data=(xtest_IncepF31, ytest_IncepF31),
	                                            steps_per_epoch=len(xtrain_IncepF31) // BS,
	                                            epochs=30,  callbacks = [callback])
    
    pred_IncepF31 = model3_IncepF31.predict(xtest_IncepF31)
    
    oos_yF31.append(ytest_IncepF31)
    oos_predF31.append(pred_IncepF31)  
    print(classification_report(ytest_IncepF31.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_IncepF31.argmax(axis=1),  
                                        target_names=le3_Incep.classes_))

#   score = metrics.accuracy_score(ytest_IncepF.argmax(axis=1),	        #ค่า acc
#                                               pred_IncepF.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(ytest_IncepF.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_IncepF.argmax(axis=1) ) )

Fold #1
Epoch 1/30
45/45 [==============================] - 207s 4s/step - loss: 0.4663 - accuracy: 0.6417 - val_loss: 0.7496 - val_accuracy: 0.4725
Epoch 2/30
45/45 [==============================] - 188s 4s/step - loss: 0.3244 - accuracy: 0.7944 - val_loss: 0.5518 - val_accuracy: 0.5934
Epoch 3/30
45/45 [==============================] - 186s 4s/step - loss: 0.2740 - accuracy: 0.8250 - val_loss: 0.5254 - val_accuracy: 0.7033
Epoch 4/30
45/45 [==============================] - 187s 4s/step - loss: 0.1984 - accuracy: 0.8917 - val_loss: 0.3224 - val_accuracy: 0.8022
Epoch 5/30
45/45 [==============================] - 191s 4s/step - loss: 0.1850 - accuracy: 0.9056 - val_loss: 0.5997 - val_accuracy: 0.7253
Epoch 6/30
45/45 [==============================] - 191s 4s/step - loss: 0.1834 - accuracy: 0.8972 - val_loss: 0.7332 - val_accuracy: 0.6813
Epoch 7/30
45/45 [==============================] - 190s 4s/step - loss: 0.1598 - accuracy: 0.8944 - val_loss: 0.6397 - val_accuracy: 0.7802
     

In [ ]:
oos_yF31 = np.concatenate(oos_yF31)
oos_predF31 = np.concatenate(oos_predF31)

In [ ]:
print(classification_report(oos_yF31.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_predF31.argmax(axis=1),  
                                        target_names=le3_Incep.classes_))

## 4) Find Tuning 
###--> ACC =  76
###--> ใช้เวลา 4447.95 วินาที
*   BS = 1
*   activation="softmax"
*   fix weight & bias in layer 0-100
*   learning_rate=0.0001
*   epoch 30







In [ ]:
train_img3_Incep = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (299, 299))                                       #ขนาดของ input
    train_img3_Incep.append(img)

data3_Incep = np.array(train_img3_Incep, dtype="float32") / 255.0

100%|██████████| 451/451 [00:05<00:00, 86.58it/s] 


In [ ]:
labels3_Incep = np.array(data_train3[['NameType']] )        
labels3_Incep.shape = (len(labels3_Incep),) 

le3_Incep = LabelEncoder()                                                                     
labels3_Incep = le3_Incep.fit_transform(labels3_Incep)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels3_Incep = to_categorical(labels3_Incep, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

##### Model

In [ ]:
base_model3_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model3_Incep.layers)

311

In [ ]:
base_model3_Incep.trainable = True
for layer in base_model3_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False

In [ ]:
#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model3_Incep.output)

prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model3_IncepF4 = tf.keras.models.Model(inputs=base_model3_Incep.input, outputs=prediction_layer)

model3_IncepF4.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
INIT_LR = 1e-1
BS = 1

aug4 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
kf_IncepF4 = KFold(5, shuffle=True, random_state=40) 

oos_yF4 = []
oos_predF4 = []
fold = 0

for train, test in kf_IncepF4.split(data3_Incep):
    fold+=1
    print(f"Fold #{fold}")
        
    xtrain_IncepF4 = data3_Incep[train]
    ytrain_IncepF4 = labels3_Incep[train]
    xtest_IncepF4 = data3_Incep[test]
    ytest_IncepF4 = labels3_Incep[test]

    M_IncepF4 = model3_IncepF4.fit( x=aug4.flow(xtrain_IncepF4, ytrain_IncepF4, batch_size=BS),
	                                            validation_data=(xtest_IncepF4, ytest_IncepF4),
	                                            steps_per_epoch=len(xtrain_IncepF4) // BS,
	                                            epochs=30,  callbacks = [callback])
    
    pred_IncepF4 = model3_IncepF4.predict(xtest_IncepF4)
    
    oos_yF4.append(ytest_IncepF4)
    oos_predF4.append(pred_IncepF4)  
    print(classification_report(ytest_IncepF4.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_IncepF4.argmax(axis=1),  
                                        target_names=le3_Incep.classes_))

#   score = metrics.accuracy_score(ytest_IncepF.argmax(axis=1),	        #ค่า acc
#                                               pred_IncepF.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(ytest_IncepF.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_IncepF.argmax(axis=1) ) )

Fold #1
Epoch 1/30
360/360 [==============================] - 249s 665ms/step - loss: 0.5995 - accuracy: 0.4694 - val_loss: 0.8508 - val_accuracy: 0.5714
Epoch 2/30
360/360 [==============================] - 240s 666ms/step - loss: 0.4782 - accuracy: 0.5861 - val_loss: 0.5635 - val_accuracy: 0.6044
Epoch 3/30
360/360 [==============================] - 238s 661ms/step - loss: 0.4199 - accuracy: 0.6667 - val_loss: 0.6837 - val_accuracy: 0.6484
Epoch 4/30
360/360 [==============================] - 239s 663ms/step - loss: 0.3905 - accuracy: 0.6917 - val_loss: 0.6726 - val_accuracy: 0.7033
Epoch 5/30
360/360 [==============================] - 289s 802ms/step - loss: 0.3414 - accuracy: 0.7472 - val_loss: 0.4193 - val_accuracy: 0.7033
Epoch 6/30
360/360 [==============================] - 239s 663ms/step - loss: 0.3496 - accuracy: 0.7667 - val_loss: 0.4928 - val_accuracy: 0.7033
Epoch 7/30
360/360 [==============================] - 237s 658ms/step - loss: 0.2890 - accuracy: 0.8333 - val_loss: 

In [ ]:
oos_yF4 = np.concatenate(oos_yF4)
oos_predF4 = np.concatenate(oos_predF4)

In [ ]:
print(classification_report(oos_yF4.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_predF4.argmax(axis=1),  
                                        target_names=le3_Incep.classes_))

              precision    recall  f1-score   support

  Lungcancer       0.83      0.70      0.76       150
      Normal       0.88      0.79      0.83       151
Tuberculosis       0.63      0.79      0.70       150

    accuracy                           0.76       451
   macro avg       0.78      0.76      0.76       451
weighted avg       0.78      0.76      0.76       451

